# Is there a rural hospital shortage in TN?
### Hospitals are big business in TN. As of 2019, 3 of the top 6 largest USA health systems are headquartered in TN (HCA, CHS, LifePoint Health), and yet there are news reports of hospitals in rural areas of Tennessee closing and leaving communities without a medical facility.
### Null Hypothesis: There is a shortage of hospitals in rural Tennessee.




In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

#! pip install folium==0.5.0
import folium # plotting library
import time
print('Folium installed')
print('Libraries imported.')
import time
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns

## Get a list of Tennessee hospitals
1. Read the webpage 
2. Use Beautiful Soup to parse the HTML
3. Pull the table data
4. Display the data to be sure we have what we need

In [ ]:
# The webiste would not allow me to connect via code, so I had to download the webpage
# The list is provided by the Tennessee Department of Safety and HomeLand Security
url='C:/Capstone/Health_Services_TN.csv'
dfhospital = pd.DataFrame
dfhospital = pd.read_csv(url)
dfhospital.head()

## Get a list of Tennesse cities with their  population


In [ ]:
# List from Wikipedia
urlcity='https://en.wikipedia.org/wiki/List_of_municipalities_in_Tennessee'
source = requests.get(urlcity).text
soup=BeautifulSoup(source,'lxml')

from IPython.display import display_html
tab = str(soup.table)
#display_html(tab,raw=True)

dfs1 = pd.read_html(tab)
dfcity =dfs1[0]
dfcity.columns=['City','County','Population','Area','Charter','Idate','Region']
dfcity.drop(['Charter','Area','Idate'],axis=1, inplace=True)
dfcity.head(5)

In [ ]:

dfhospital.shape


## Map hospitals

In [ ]:
tnlatitude= 35.5175
tnlongitude = -86.5804
map_tn = folium.Map(location=[tnlatitude, tnlongitude], zoom_start=11)

# add markers to map

for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tn)  
map_tn

In [ ]:
urlcounty='https://en.wikipedia.org/wiki/List_of_Tennessee_locations_by_per_capita_income'
source = requests.get(urlcounty).text
soup=BeautifulSoup(source,'lxml')

from IPython.display import display_html
tab = str(soup.table)
#display_html(tab,raw=True)

dfs2 = pd.read_html(tab)
dfcounty =dfs2[0]
dfcounty.columns=['Rank','County','Per_Cap_Income','Median_House_Income','Median_Family_Income','Population','Households']
# Tennessee and United States are in the data, so we need to drop them from the list
# dfcounty.dropna(subset=['Rank'],inplace=True)
dfcounty= dfcounty[dfcounty['Rank'].notna()]


dfcounty.head()
# Convert to number

dfcounty['Per_Cap_Income']=dfcounty['Per_Cap_Income'].str.replace('$','').str.replace(',','').astype(float)
dfcounty['Median_House_Income']=dfcounty['Median_House_Income'].str.replace('$','').str.replace(',','').astype(float)
dfcounty['Median_Family_Income']=dfcounty['Median_Family_Income'].str.replace('$','').str.replace(',','').astype(float)
dfcounty


In [ ]:
df=pd.merge(dfcounty,dfhospital, on='County',how='left')

In [ ]:

df.head()

In [ ]:
#Drop some columns
df = df.drop(['X','Y','HospitalID','Expr1','EPCPhone','ESRI_OID'],axis=1)

In [ ]:
#Drop some more columns
df=df.drop(['RadioCode1','EPC','ebola_ttx','GlobalID','NDMSFacility','DMATTeamSite','RadioCode1'],axis=1)

In [ ]:
# Get USA county json
#From Census.gov
with open('c:/Capstone/TNCounties.json') as usa:
    data=usa.read()
    

usa_geo = json.loads(data)
tn_geo = usa_geo
print ('Download complete')




In [ ]:
df.head()

In [ ]:
#Is County rural or not based on US Department of Agriculture Rural-Urban Continuum Code.
url='C:/Capstone/ruralurbancodes2013.xls'
dfrural = pd.DataFrame
dfrural = pd.read_excel(url)
#Filter for TN
dftnco=dfrural[dfrural["State"]=='TN']
dftnco.rename(columns ={"County_Name":"County","RUCC_2013":"RUCC"},inplace=True)
dftnco.head()

# County map with hospitals based upon county population

In [ ]:
df1=pd.merge(dfcounty,dfhospital, on='County',how='left')
dfco=pd.merge(df1,dftnco, on='County',how='left')

In [ ]:
#An Expr1==0 means no hospial in the county. Expr1==1 means a hospital is in the county
dfco["X"]= dfco["X"].fillna(0)
dfco["Y"]= dfco["Y"].fillna(0)
dfco = dfco.replace(np.nan,'',regex=True)
#dfco.drop(['HospitalID','Status','Factype','FacilityType','Address3','RegionType','NDMSFacility','DMATTeamSite'],axis=1, inplace=True)
#dfco.drop(['RadioCode1','EPC','EPCPhone','ESRI_OID','ebola_ttx','GlobalID'],axis=1, inplace=True)
dfco.head()

In [ ]:
dfco['Hospital']=np.where(dfco["Expr1"]==1,True,False)
dfcon = dfco.filter(['Hospital','County','Per_Cap_Income','Median_House_Income','Median_Family_Income','Population','RUCC'])
dfcon=dfcon.drop_duplicates(subset=['County'])
dfcon.head()

In [ ]:
dfcon.describe()

In [ ]:
dfwith=dfcon[dfcon['Hospital']==True]
dfwith.describe()

In [ ]:
dfwithout=dfcon[dfcon['Hospital']==False]
dfwithout.describe()

In [ ]:
dfcon.sort_values(by=['County'])
dfcon.shape                   

In [ ]:
#Pouplation seems to correspond to not having a hospital
sns.boxplot(x="Hospital",y="Population",data=dfcon)

# RUCC Code
### 9 - Nonmetro - Completely rural or less than 2,500 urban population, not adjacent to a metro area                                                                                                           
### 8 - Nonmetro - Completely rural or less than 2,500 urban population, adjacent to a metro area                                   
                                                                                                          
### 7 - Nonmetro - Urban population of 2,500 to 19,999, not adjacent to a metro area                                                                                                                                                                     
### 6 - Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area                                                                                                                                                                            
### 5- Nonmetro - Urban population of 20,000 or more, adjacent to a metro area                                                                                                                                 
### 4- Nonmetro - Urban population of 20,000 or more, adjacent to a metro area                                                                                                                                 
### 3 - Metro - Counties in metro areas                                                                                                                            
                                                                                                                     


In [ ]:
# An RUCC of 3 signifies large metro areas
sns.boxplot(x="Hospital",y="RUCC",data=dfcon)

In [ ]:
sns.boxplot(x="Hospital",y="Per_Cap_Income",data=dfcon)

In [ ]:
sns.boxplot(x="Hospital",y="Median_House_Income",data=dfcon)

In [ ]:
from scipy import stats
pearson_coef, p_value = stats.pearsonr(dfcon['Population'], dfcon['Hospital'])

print("The Pearson Correlation Coefficient of Population is", pearson_coef, " with a P-value of P =", p_value)  

In [ ]:
pearson_coef, p_value = stats.pearsonr(dfcon['Per_Cap_Income'], dfcon['Hospital'])

print("The Pearson Correlation Coefficient of Per Capita Income is", pearson_coef, " with a P-value of P =", p_value)  

In [ ]:
pearson_coef, p_value = stats.pearsonr(dfcon['RUCC'], dfcon['Hospital'])

print("The Pearson Correlation Coefficient of Rural Code is", pearson_coef, " with a P-value of P =", p_value)  

In [ ]:
pearson_coef, p_value = stats.pearsonr(dfcon['Median_House_Income'], dfcon['Hospital'])

print("The Pearson Correlation Coefficient of Median Household Income is", pearson_coef, " with a P-value of P =", p_value)  

In [ ]:
dfcon.describe()

In [ ]:
#ANOVA
grouped_test2=dfcon[['Hospital','RUCC', 'Per_Cap_Income','Median_House_Income','Population']].groupby(['Hospital'])
grouped_test2.describe()
# ANOVA
f_val, p_val = stats.f_oneway(grouped_test2.get_group(False)['RUCC'], grouped_test2.get_group(True)['RUCC'])  
 
print( "ANOVA results for RUCC: F=", f_val, ", P =", p_val)   

In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group(False)['Population'], grouped_test2.get_group(True)['Population'])  
 
print( "ANOVA results for Population: F=", f_val, ", P =", p_val)   

In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group(False)['Median_House_Income'], grouped_test2.get_group(True)['Median_House_Income'])  
 
print( "ANOVA results for Median Household Income: F=", f_val, ", P =", p_val)   

In [ ]:
plt.figure(figsize=(8,5))
x_data, y_data = (dfcon["RUCC"].values, dfcon["Population"].values)
plt.plot(x_data, y_data, 'ro')
plt.ylabel('Population')
plt.xlabel('RUCC')
plt.show()

In [ ]:

tn_map = folium.Map(location=[tnlatitude,tnlongitude], zoom_start=7)
# generate choropleth map using the population of each TN county
 
tn_map.choropleth(
    geo_data=tn_geo,
    data=dfcounty,
    columns=['County','Population'],
    key_on='feature.properties.NAME',
    threshold_scale=[15000,100000,200000,400000,1000000],
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='TN County Population'
)
for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_map) 

# display map
tn_map

In [ ]:
tn_map = folium.Map(location=[tnlatitude,tnlongitude], zoom_start=7)
# generate choropleth map using the population of each TN county
 
tn_map.choropleth(
    geo_data=tn_geo,
    data=dfcounty,
    columns=['County','Median_House_Income'],
    key_on='feature.properties.NAME',
    threshold_scale=[15000,30000,40000,100000],
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='TN County Median Household Income'
)
for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_map) 

# display map
tn_map

In [ ]:
#We only need TN counties
dfrural = dfrural[dfrural['State']=='TN']
dfrural.head(10)

# The USDA provides a classification of counties.
## A category code <=3 is consider an urban area.
## A category code of 8 or 9 represents rural areas that may or may not be adjacent to metro areas

In [ ]:
tn_map2 = folium.Map(location=[tnlatitude,tnlongitude], zoom_start=7)
# generate choropleth map using the population of each TN county
 
tn_map2.choropleth(
    geo_data=tn_geo,
    data=dfrural,
    columns=['County_Name','RUCC_2013'],
    key_on='feature.properties.NAME',
    #threshold_scale=[3,4,6,7,8,9],
    
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='TN Urban to Rural Categorization'
)
for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_map2) 

# display map
tn_map2

# The above maps would indicate that rural/ low population counties do have a lack of hospitals. No Tennesse county that is classified as rural and not adjacent to a metro area has a hospital located in it.

In [ ]:
dfcon.head()

## Logistic regression

In [ ]:
dfcon1 = dfcon[['Hospital','Per_Cap_Income','Median_House_Income','Population','RUCC']]
dfcon1['Hospital'] = dfcon1['Hospital'].astype('int')
dfcon1.head()

In [ ]:
X = np.asarray(dfcon1[['RUCC']])
#X = np.asarray(dfcon1[['Per_Cap_Income','Median_House_Income','Population','RUCC']])
y = np.asarray(dfcon1['Hospital'])
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
yhat = LR.predict(X_test)
yhat

In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat,pos_label=0)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

In [ ]:
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Hospital=1','Hospital=0'],normalize= False,  title='Confusion matrix')

In [ ]:
print (classification_report(y_test, yhat))

## SVC

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 
yhat = clf.predict(X_test)
yhat [0:5]
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    


In [ ]:
    # Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot non-normalized confusion matrix

plot_confusion_matrix(cnf_matrix, classes=['Hospital=1','Hospital=0'],normalize= False,  title='Confusion matrix')

#  Counties display on a Per Capita Income basis

In [ ]:
tn_mapd = folium.Map(location=[tnlatitude,tnlongitude], zoom_start=7)
# generate choropleth map using the population of each TN county
 
tn_mapd.choropleth(
    geo_data=tn_geo,
    data=dfcounty,
    columns=['County','Per_Cap_Income'],
    key_on='feature.properties.NAME',
    threshold_scale=[12000,17000,22000,30000],
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    
    legend_name='TN County Per Capita Income'
)
for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_mapd) 

# display map
tn_mapd

### As seen in the chart above, and the numbers below, we will need to group the couties by a population category.

In [ ]:
#Identify counties with out a hospital
pdnone=df[df['HospitalName'].isnull()]

In [ ]:
pdnone.describe()

In [ ]:
#What is the best indicator of not having a hospital?
viz = df[['Population','Per_Cap_Income','Median_House_Income','Households']]
viz.hist()
plt.show()

In [ ]:
tn_mapd = folium.Map(location=[tnlatitude,tnlongitude], zoom_start=7)
# generate choropleth map using the population of each TN county
 
tn_mapd.choropleth(
    geo_data=tn_geo,
    data=dfcounty,
    columns=['County','Population'],
    key_on='feature.properties.NAME',
    threshold_scale=[15000,100000,200000,400000,1000000],
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    
    legend_name='TN County Per Population Group'
)
for lat, lng, name, city in zip(dfhospital['Latitude'], dfhospital['Longitude'],dfhospital['HospitalName'],dfhospital['City']):
    label = '{}, {}'.format(name, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_mapd) 

# display map
tn_mapd

## Use a For loop and Folium to to put our neighborhoods on the map

In [ ]:
dfzip =pd.DataFrame()
dfzip=pd.read_excel('c:/Capstone/USA Cities.xlsx')
dfzip.head()

In [ ]:
#Only show TN cities

dftncity=dfzip[dfzip['State']=='TN']
#local = ['Algood','Baxter','Celina']
#dftncity=dftncity[dftncity.City.isin(local)]

In [ ]:
dftncity

In [ ]:
#dftncity['City'] = dftncity['City'].str.upper()
pdc = pd.DataFrame
pdc = pd.merge(dftncity,dfhospital, on='City', how="left")

In [ ]:
pdc

In [ ]:
#This could Drop rows that have a hospital, but to get average distances we should keep them
#dfc = pdc[pdc['X'].isna()]
dfc = pdc

In [ ]:
# No duplcate cites are needed since distance difference should be only a few KMs.
#dfc = dfc.drop_duplicates(subset=["City"])
dfc

# Find the distance to the nearest hospital

In [ ]:
#Create function to calculate distance
from sklearn.metrics.pairwise import haversine_distances
from math import radians
dfhospital.head()

In [ ]:
#Convert Lat and Long to float 
dfhospital['X'] = dfhospital['X'].apply(float)
dfhospital['Y'] = dfhospital['Y'].apply(float)

## Build function to calulate distance
### uses miles

In [ ]:
def haversine_vectorize(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    newlon = lon2 - lon1
    newlat = lat2 - lat1

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 3958 * dist #6367 for distance in KM for miles use 3958
    return km

In [ ]:
# Loop through the city list and then loop through the hospital list to find the distance.
dist_list = []


for index, crow in dfc.iterrows():

    for index2, chos in dfhospital.iterrows():
        
        result = haversine_vectorize(crow['Longitude_x'],crow['Latitude_x'],chos['X'],chos['Y'] )
       
        dict1 = {}
        dict1.update({'CityLat': crow['Latitude_x'],'CityLng': crow['Longitude_x'],'CityCity': crow['City'],'HospitalLat': chos['Latitude'],'HospitalLng': chos['Longitude'],'HospitalCity': chos['City'],'Distance': result, 'Hospital':chos['HospitalName']})
        dist_list.append(dict1)                           
        
dfdistance = pd.DataFrame(dist_list)                                   
                    



In [ ]:
dfdistance.head()

In [ ]:
#dfdistance.set_index(['CityCity'],inplace=True)
dfdistance.reset_index()


In [ ]:
#We only need to see the closet hospital
dfmin=pd.DataFrame
dfmin=dfdistance.reset_index()
dfm=dfmin.groupby('CityCity').Distance.idxmin()
dfm=pd.DataFrame(dfm,columns=['Distance'])


In [ ]:
dfcmin = pd.DataFrame()


for index , x in dfm.iterrows():
    row = x['Distance']
    dfcmin = dfcmin.append(dfdistance.iloc[x])
    
    

In [ ]:
 dfmm = dfcmin.sort_values(['Distance'],ascending=False)
dfmm.describe()

# DO Not use for now
### Use Foursquare to get distances to a hospital
###  Foursquare returns venues based upon populatiry and not distance. Stupdi, but that's the way it is.

df = pd.DataFrame()

for index, row in dfc.iterrows():
    geocode = row['geopoint'] 
    
    
    time.sleep(1)
    # Used for FourSquare
    address = get_nearest_hospital(row['City'],geocode)
    
        
    print(row['City'] + '  -  ' + str(address['meta']['code']))
    
    if address['meta']['code']==200:
        
            #print (address)
            venues = address['response']['venues']
            df = df.append(pd.json_normalize(venues),ignore_index=True)
            #df['referralId'].iloc[-1]=row['City']
    else:
            x = row['City']
            x

In [ ]:
df.head()
